In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
exp_dir = Path('experiments')
data_dir = exp_dir / 'all_targets'
motif_dir = data_dir / 'motif_fits'

In [31]:
motif_target_dis_dict = {}
motif_folders = list(motif_dir.glob('*'))
for folder in motif_folders:
    motif_name = folder.name
    print("Motif: ", motif_name)
    motif_target_dis_dict[motif_name] = {}
    result_files = list(folder.glob('*.npz'))
    for result_file in result_files:
        target_name = result_file.name.split('.')[0][8:]  #skip 'result_'
        print("Target: ", target_name) 
        data = np.load(result_file)
        motif_target_dis_dict[motif_name][target_name] = {
            'train_error': data['training_loss'],
            'test_error': data['test_loss'],
            'jac_fro': data['jac_fro']
        }

Motif:  bibla
Motif:  bistable
Motif:  lc
Target:  bz_reaction
Motif:  lds
Target:  bz_reaction
Motif:  ring
Motif:  sphere
Target:  lorenz_on_313


In [32]:
motif_target_dis_dict

{'bibla': {},
 'bistable': {},
 'lc': {'bz_reaction': {'train_error': array(0.26460284),
   'test_error': array(0.38057023),
   'jac_fro': array(4.5148945, dtype=float32)}},
 'lds': {'bz_reaction': {'train_error': array(0.07649617),
   'test_error': array(0.20433637),
   'jac_fro': array(1.555247, dtype=float32)}},
 'ring': {},
 'sphere': {'lorenz_on_313': {'train_error': array(0.42178538),
   'test_error': array(0.44765702),
   'jac_fro': array(0.2544364, dtype=float32)}}}